# Crea il tuo assistente virtuale
In questa esercitazione dovrai implementare un meccanismo di memoria per trasformare un LLM in un assistente virtuale in grado di parlare in italiano. Scegli tu quale LLM ritieni più opportuno utilizzare.

### Suggerimenti
- Il modo più semplice per implementare la memoria è fare in modo che il modello riceva in input tutta la conversazione come prompt ogni volta, sia le domande poste dall'utente che le risposte delle assistente, modelli diversi hanno standard diversi per la codifica di domande e risposte, ti consiglio di dare uno sguardo [a questo](https://huggingface.co/docs/transformers/chat_templating)
- Per fare in modo che l'assistente sia in grado di dialogare in italiano puoi cercare un LLM in grado di chattare in italiano, oppure puoi utilizzare 2 modelli di language translation per tradurre l'input dell'utente in inglese e poi la risposta dell'utente in italiano, ricordà però che la memoria dell'assistente deve essere nella sua lingua originale.

## Soluzione

In [ ]:
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.1 MB/s eta 0:00:00


In [ ]:
import transformers
import torch

## Soluzione base

In [ ]:
CHAT_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
EN_IT_MODEL_NAME = "Helsinki-NLP/opus-mt-en-it"
IT_EN_MODEL_NAME = "Helsinki-NLP/opus-mt-it-en"

In [ ]:
chat_tokenizer = transformers.AutoTokenizer.from_pretrained(CHAT_MODEL_NAME)
chat_model = transformers.AutoModelForCausalLM.from_pretrained(CHAT_MODEL_NAME)

en_it_tokenizer = transformers.AutoTokenizer.from_pretrained(EN_IT_MODEL_NAME)
en_it_model = transformers.MarianMTModel.from_pretrained(EN_IT_MODEL_NAME)

it_en_tokenizer = transformers.AutoTokenizer.from_pretrained(IT_EN_MODEL_NAME)
it_en_model = transformers.MarianMTModel.from_pretrained(IT_EN_MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/344M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
text = ""
history = []

while text.lower()!="addio":

  text = input("Tu: ")

  tokens = it_en_tokenizer.encode(text, return_tensors="pt")
  output = it_en_model.generate(tokens)
  prompt = it_en_tokenizer.decode(output[0], skip_special_tokens=True)

  history.append({"role":"user","content":prompt})

  tokens = chat_tokenizer.apply_chat_template(history,return_tensors="pt")
  output = chat_model.generate(tokens, max_new_tokens=100, pad_token_id=100)
  answer = chat_tokenizer.decode(output[0], skip_special_tokens=True)
  answer = answer.split("[/INST]")[-1].strip()

  history.append({"role":"assistant","content":answer})

  tokens = en_it_tokenizer.encode(answer, return_tensors="pt")
  output = en_it_model.generate(tokens)
  translation = en_it_tokenizer.decode(output[0], skip_special_tokens=True)

  print("Assistant: "+translation)

Assistant: Piacere di conoscerti, Giuseppe! Puoi dirmi di più sulla ProfessioneAI e la sua missione?
Assistant: E 'comprensibile che si può considerare la pensione ad un certo punto, ma in ultima analisi la decisione di andare in pensione o non dovrebbe essere basata sulle vostre circostanze personali e obiettivi. Se si sente che si è ancora appassionato per il vostro lavoro e hanno l'energia e la spinta a continuare a leader ProfessioneAI, allora può essere la pena di considerare di mantenere il vostro ruolo di CEO. Tuttavia, se si decide di andare in pensione, è importante avere un piano chiaro in atto per il futuro della società.


KeyboardInterrupt: Interrupted by user

## Soluzione avanzata: creiamo una classe

In [ ]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
import json

class Assistant:

  _SYSTEM_PROMPT = "Sei un chatbot amichevole che parla italiano, le tue risposte devono essere in italiano, no in inglese."

  def __init__(self, model_name):
    self._pipeline = transformers.pipeline("text-generation", model=model_name, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")
    self._history = [{"role":"system", "content":self._SYSTEM_PROMPT}]


  def ask(self, text):
    prompt = self._history
    prompt.append({"role":"user","content":text})
    output = self._pipeline(prompt)
    response = output[0]["generated_text"]
    self._history = response
    return response[-1]["content"]



assistant = Assistant(model_name=MODEL_NAME)
print(assistant.ask("Io sono Giuseppe"))
print(assistant.ask("Qual è il mio nome?"))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  Ciao Giuseppe! Come stai oggi? 😊 Un chatbot amichevole come me è qui per parlare con te e rispondere alle tue domande. Qualcosa che ti chiedi o qualcosa che vuoi dire? 🤔
  Ciao Giuseppe! Il tuo nome è... *adotta un tono di voce esagerato*...Giuseppe! 😄 Sono felice di conoscerti, cosa vuoi chiedere o parlare? 🤔
